### 필요한 패키지 Import
DataFrame의 내용을 생략없이 출력하려면 아래 주석처리한 부분의 주석을 해제하고 실행

In [1]:
import os

import numpy as np
import pandas as pd

from collections import Counter

# pd.set_option('display.max_colwidth', None)

### 변수 정의

In [2]:
DATAPATH = "../../data/"
DATAFILE = os.path.join(DATAPATH, f"train.csv")
SOURCE_DATA = pd.read_csv(DATAFILE)
data = SOURCE_DATA.copy()

In [3]:
data

,ID,input_text,label_text,target,predefined_news_category,annotations,url,date
0,ynat-v1_train_00000,유튜브 내달 2일까지 크리에이터 지원 공간 운영,생활문화,3,IT과학,"{'annotators': ['08', '13', '07'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.06.30. 오전 10:36
1,ynat-v1_train_00001,어버이날 막따가 흐려저…남부지방 여튼 황사,생활문화,3,생활문화,"{'annotators': ['14', '13', '07'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.05.08. 오전 5:25
2,ynat-v1_train_00001,어버이날 맑다가 흐려져…남부지방 옅은 황사,생활문화,3,생활문화,"{'annotators': ['14', '13', '07'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.05.08. 오전 5:25
3,ynat-v1_train_00002,내년부터 국가RD 평가 때 논문건수는 반영 않는다,사회,2,IT과학,"{'annotators': ['14', '13', '10'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.03.15. 오후 12:00
4,ynat-v1_train_00003,김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것,사회,2,IT과학,"{'annotators': ['16', '11', '15'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2017.02.28. 오전 9:54
...,...,...,...,...,...,...,...,...
52525,ynat-v1_train_45673,사우디 아람코 주요 석유시설·유전 드론 공격에 화재,세계,4,세계,"{'annotators': ['14', '07', '08'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2019.09.14. 오후 4:36
52526,ynat-v1_train_45674,박기원 감독 눈치 보지 말고…비예나 눈치 본 건 아닌데,스포츠,6,스포츠,"{'annotators': ['09', '13', '18'], 'annotation...",https://sports.news.naver.com/news.nhn?oid=001...,2019.11.03 17:06
52527,ynat-v1_train_45675,아시안게임 첫 경기 앞둔 라건아 인도네시아 팬이라도 ...,스포츠,6,스포츠,"{'annotators': ['07', '08', '12'], 'annotation...",https://sports.news.naver.com/news.nhn?oid=001...,2018.08.13 17:54
52528,ynat-v1_train_45676,트럼프 미중 무역협상 1단계 합의 서명식 가질 것종합,세계,4,세계,"{'annotators': ['07', '18', '02'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2019.12.25. 오후 4:36


### G2P Augmented Data Detection

G2P 증강 기법을 활용하여 증강한 데이터를 먼저 확인한다.

AI Stages의 학습데이터 설명 중 G2P 증강에 대한 내용은 다음과 같다.
> 노이즈 데이터의 경우 전체 학습 데이터의 15%(총 6,852개)에 해당합니다. noise data 중 80%(5,481개)는 G2P를 이용한 text perturbation으로 생성되었으며, prescriptive pronunciation과 descriptive pronunciation을 1:1 비율로 사용하였 noise를 생성하였습니다.

먼저 raw data를 관찰하여 G2P 데이터의 분포를 확인한다. \
G2P로 증강한 데이터는 고유한 ID를 갖지 않고, 원본 데이터의 ID를 따라가므로, 중복된 ID를 갖는 모든 데이터를 뽑아낸 후, \
G2P로 증강한 데이터만 제거한다.

여기서는 중복된 ID를 갖는 데이터 쌍에서 인덱스가 더 작은 데이터가 G2P 증강 데이터, \
인덱스가 큰 데이터를 원본이라고 가정하고, 인덱스가 더 작은 데이터를 모두 제거하였음.

In [4]:
data.iloc[1:3]

,ID,input_text,label_text,target,predefined_news_category,annotations,url,date
1,ynat-v1_train_00001,어버이날 막따가 흐려저…남부지방 여튼 황사,생활문화,3,생활문화,"{'annotators': ['14', '13', '07'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.05.08. 오전 5:25
2,ynat-v1_train_00001,어버이날 맑다가 흐려져…남부지방 옅은 황사,생활문화,3,생활문화,"{'annotators': ['14', '13', '07'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.05.08. 오전 5:25


In [5]:
# 기존 코드 (폐기)

# g2p_augmented = data["ID"].duplicated()
# duplicated_data_idx = []
# for i, bool in enumerate(g2p_augmented):
#     if bool:
#         duplicated_data_idx.append(i - 1)

# len(duplicated_data_idx), duplicated_data_idx[:10]

# g2p_data = data.reset_index()
# g2p_data = g2p_data[g2p_data["index"].isin(duplicated_data_idx)].drop("index", axis=1)
# g2p_data

# g2p_removed_data = data.reset_index()
# g2p_removed_data = g2p_removed_data[~g2p_removed_data["index"].isin(duplicated_data_idx)].drop("index", axis=1)
# g2p_removed_data

In [6]:
duplicated_data_idx = data[data[["ID", "label_text"]].duplicated()].index - 1
# duplicated_data_idx = data[data["ID"].duplicated()].index - 1
g2p_data = data.loc[duplicated_data_idx]
g2p_data

,ID,input_text,label_text,target,predefined_news_category,annotations,url,date
1,ynat-v1_train_00001,어버이날 막따가 흐려저…남부지방 여튼 황사,생활문화,3,생활문화,"{'annotators': ['14', '13', '07'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.05.08. 오전 5:25
5,ynat-v1_train_00004,회새긴간 작까 김동시 걍심꼬백 뜽 새 소설집 뚜권 출간,생활문화,3,생활문화,"{'annotators': ['03', '02', '12'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2018.04.03. 오전 7:05
9,ynat-v1_train_00007,미세먼지 속 출근길,생활문화,3,사회,"{'annotators': ['03', '10', '09'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.17. 오전 10:14
43,ynat-v1_train_00040,월드커 파나은행 대표티메 행우늬 이달러 이영영장 선물,스포츠,6,스포츠,"{'annotators': ['10', '16', '08'], 'annotation...",https://sports.news.naver.com/news.nhn?oid=001...,2018.06.12 13:04
66,ynat-v1_train_00061,서규왕국 싸우디 태양광·풍녁 빨쩐 중심지 될 껃,세계,4,세계,"{'annotators': ['12', '14', '13'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2019.01.16. 오후 7:38
...,...,...,...,...,...,...,...,...
52462,ynat-v1_train_45616,朴대통녕 궁미니 구쾨서 외면당하면 직쩜 나설수바께,정치,0,정치,"{'annotators': ['12', '03', '15'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.01.19. 오전 11:02
52466,ynat-v1_train_45619,L지유플러스 오지 통하평 광설로 감시시스템 개발,IT과학,5,IT과학,"{'annotators': ['15', '12', '16'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2018.12.04. 오전 10:27
52474,ynat-v1_train_45626,금늉연구워 놀해 성장뉼 전망치 이.육→이.사퍼센트 하향조정,경제,1,경제,"{'annotators': ['14', '09', '07'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2019.05.23. 오후 12:00
52495,ynat-v1_train_45646,일본 재정 흑짜 달성 목표 이처니시븅녀느로 재수정,세계,4,세계,"{'annotators': ['07', '12', '09'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2019.01.31. 오전 9:49


In [7]:
non_duplicated_data_idx = data.index.difference(duplicated_data_idx)
g2p_removed_data = data.loc[non_duplicated_data_idx]
g2p_removed_data

,ID,input_text,label_text,target,predefined_news_category,annotations,url,date
0,ynat-v1_train_00000,유튜브 내달 2일까지 크리에이터 지원 공간 운영,생활문화,3,IT과학,"{'annotators': ['08', '13', '07'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.06.30. 오전 10:36
2,ynat-v1_train_00001,어버이날 맑다가 흐려져…남부지방 옅은 황사,생활문화,3,생활문화,"{'annotators': ['14', '13', '07'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.05.08. 오전 5:25
3,ynat-v1_train_00002,내년부터 국가RD 평가 때 논문건수는 반영 않는다,사회,2,IT과학,"{'annotators': ['14', '13', '10'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.03.15. 오후 12:00
4,ynat-v1_train_00003,김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것,사회,2,IT과학,"{'annotators': ['16', '11', '15'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2017.02.28. 오전 9:54
6,ynat-v1_train_00004,회색인간 작가 김동식 양심고백 등 새 소설집 2권 출간,생활문화,3,생활문화,"{'annotators': ['03', '02', '12'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2018.04.03. 오전 7:05
...,...,...,...,...,...,...,...,...
52525,ynat-v1_train_45673,사우디 아람코 주요 석유시설·유전 드론 공격에 화재,세계,4,세계,"{'annotators': ['14', '07', '08'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2019.09.14. 오후 4:36
52526,ynat-v1_train_45674,박기원 감독 눈치 보지 말고…비예나 눈치 본 건 아닌데,스포츠,6,스포츠,"{'annotators': ['09', '13', '18'], 'annotation...",https://sports.news.naver.com/news.nhn?oid=001...,2019.11.03 17:06
52527,ynat-v1_train_45675,아시안게임 첫 경기 앞둔 라건아 인도네시아 팬이라도 ...,스포츠,6,스포츠,"{'annotators': ['07', '08', '12'], 'annotation...",https://sports.news.naver.com/news.nhn?oid=001...,2018.08.13 17:54
52528,ynat-v1_train_45676,트럼프 미중 무역협상 1단계 합의 서명식 가질 것종합,세계,4,세계,"{'annotators': ['07', '18', '02'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2019.12.25. 오후 4:36


위에서 언급한 대로 G2P로 증강한 데이터가 5,481개임을 확인하였다.

다만 여기서 적용한 규칙이 정확하지 않을 수 있으니 설명을 위해서 \
나중에 다른 방법을 사용하여 G2P 증강 여부룰 가려내는 알고리즘이 필요할 수 있다.

### CSV 파일 저장

G2P 증강 데이터를 제거한 데이터와 G2P 증강 데이터만을 모은 데이터를 구별하여 별도의 파일로 저장해준다.

In [8]:
g2p_data.to_csv("../../data/preprocessed/train_g2p_only.csv", index=False)
g2p_removed_data.to_csv("../../data/preprocessed/train_g2p_removed.csv", index=False)